Run the following in Terminal to create a virtual environment for this tutorial

In [ ]:
# Make virtual environment. Tested on python3.6+. Run in terminal, not in Jupyter.
python3.7 -m venv tutorial-env
source tutorial-env/bin/activate

# Install COBS requirements
pip3 install -r requirements.txt

# point jupyter to virtualenv 
pip3 install ipykernel
python -m ipykernel install --user --name=tutorial_env

# Now, restart jupyter notebook and change the Kernel by selecting Kernel > Change kernel > tutorial_env

For the first tutorial, we choose to run an experiment where we increase the size of the dataset to see how sample efficient the methods are. The environment is the Graph environment with a horizon of 4 and a tabular function class for the Q functions.

In [ ]:
# Imports
import numpy as np
import json
from copy import deepcopy

from ope.envs.graph import Graph
from ope.policies.basics import BasicPolicy

from ope.experiment_tools.experiment import ExperimentRunner, analysis
from ope.experiment_tools.config import Config
from ope.experiment_tools.factory import setup_params

In [ ]:
# Get configuration
configuration_filename = "tabular_experiment_cfg.json"
with open('cfgs/{0}'.format(configuration_filename), 'r') as f:
    param = json.load(f)

In [ ]:
param = setup_params(param) # Setup parameters
runner = ExperimentRunner() # Instantiate a runner for an experiment

In [ ]:
# run 5 experiments, each with a varying number of trajectories
for N in range(5):
    
    configuration = deepcopy(param['experiment']) # Make sure to deepcopy as to never change original
    configuration['num_traj'] = 8*2**N # Increase dataset size

    # store these credentials in an object
    cfg = Config(configuration)

    # initialize environment with the parameters from the config file.
    # If you'd like to use a different environment, swap this line
    env = Graph(make_pomdp=cfg.is_pomdp,
                number_of_pomdp_states=cfg.pomdp_horizon,
                transitions_deterministic=not cfg.stochastic_env,
                max_length=cfg.horizon,
                sparse_rewards=cfg.sparse_rewards,
                stochastic_rewards=cfg.stochastic_rewards)

    # set seed for the experiment
    np.random.seed(cfg.seed)

    # processor processes the state for storage,  {(processor(x), a, r, processor(x'), done)}
    processor = lambda x: x

    # absorbing state for padding if episode ends before horizon is reached. This is environment dependent.
    absorbing_state = processor(np.array([env.n_dim - 1]))

    # Setup policies. BasicPolicy takes the form [P(a=0), P(a=1), ..., P(a=n)]
    # For different policies, swap in here
    actions = [0, 1]
    pi_e = BasicPolicy(
        actions, [max(.001, cfg.eval_policy), 1 - max(.001, cfg.eval_policy)])
    pi_b = BasicPolicy(
        actions, [max(.001, cfg.base_policy), 1 - max(.001, cfg.base_policy)])

    # add env, policies, absorbing state and processor
    cfg.add({
        'env': env,
        'pi_e': pi_e,
        'pi_b': pi_b,
        'processor': processor,
        'absorbing_state': absorbing_state
    })
    cfg.add({'models': param['models']})


    # Add the configuration
    runner.add(cfg)

In [ ]:
# Run the experiments
results = runner.run()

In [ ]:
# Analyze the results
# Each row in the result is (OPE estimator, V(pi_e), MSE Error from on-policy: (V(pi_e) - True)**2)
for num, result in enumerate(results):
    print('Result Experiment %s' % (num+1))
    analysis(result)
    print('*'*20)
    print()

For the second tutorial, we choose to run the same experiment as before but with a different environment and different Q function class. The environment is the Pixel-Gridworld (Pix-GW) environment with a horizon of 5 and a NN function class for the Q functions.

In [ ]:
from ope.envs.gridworld import Gridworld
from ope.policies.epsilon_greedy_policy import EGreedyPolicy
from ope.policies.tabular_model import TabularPolicy

In [ ]:
# Get configuration
configuration_filename = "experiment_cfg.json"
with open('cfgs/{0}'.format(configuration_filename), 'r') as f:
    param = json.load(f)

In [ ]:
param = setup_params(param)
runner = ExperimentRunner() # make new runner

In [ ]:
for N in range(5):
    configuration = deepcopy(param['experiment']) # Make sure to deepcopy as to never change original
    configuration['num_traj'] = 8*2**N # Increase dataset size

    # store these credentials in an object
    cfg = Config(configuration)

    # initialize environment with the parameters from the config file.
    env = Gridworld(slippage=.2*cfg.stochastic_env)

    # Set seed and 
    np.random.seed(cfg.seed)
    eval_policy = cfg.eval_policy
    base_policy = cfg.base_policy

    # to_grid and from_grid are particular to Gridworld
    # These functions are special to convert an index in a grid to an 'image'
    def to_grid(x, gridsize=[8, 8]):
        x = x.reshape(-1)
        x = x[0]
        out = np.zeros(gridsize)
        if x >= 64:
            return out
        else:
            out[x//gridsize[0], x%gridsize[1]] = 1.
        return out

    # This function takes an 'image' and returns the position in the grid
    def from_grid(x, gridsize=[8, 8]):
        if len(x.shape) == 3:
            if np.sum(x) == 0:
                x = np.array([gridsize[0] * gridsize[1]])
            else:
                x = np.array([np.argmax(x.reshape(-1))])
        return x

    # processor processes the state for storage,  {(processor(x), a, r, processor(x'), done)}
    processor = lambda x: x
    
    # Set up e-greedy policy using epsilon-optimal
    policy = env.best_policy()
    
    # absorbing state for padding if episode ends before horizon is reached. This is environment dependent.
    absorbing_state = processor(np.array([len(policy)]))

    # Setup policies.
    pi_e = EGreedyPolicy(model=TabularPolicy(policy, absorbing=absorbing_state), processor=from_grid, prob_deviation=eval_policy, action_space_dim=env.n_actions)
    pi_b = EGreedyPolicy(model=TabularPolicy(policy, absorbing=absorbing_state), processor=from_grid, prob_deviation=base_policy, action_space_dim=env.n_actions)

    cfg.add({
        'env': env,
        'pi_e': pi_e,
        'pi_b': pi_b,
        'processor': processor,
        'absorbing_state': absorbing_state,
        'convert_from_int_to_img': to_grid, # if environment state is an int, can convert to image through this function
    })
    cfg.add({'models': param['models']})

    runner.add(cfg)

In [ ]:
# Run the experiments
results = runner.run()

In [ ]:
# Analyze the results
# Each row in the result is (OPE estimator, V(pi_e), MSE Error from on-policy: (V(pi_e) - True)**2)
for num, result in enumerate(results):
    print('Result Experiment %s' % (num+1))
    analysis(result)
    print('*'*20)
    print()